## Part 3: Adsorption
Assistants: Xing Wang & Moloud Kaviani

Room:	N216

Phone:	031 631 56 25

Email:	xing.wang@dcb.unibe.ch, moloud.kaviani@dcb.unibe.ch  

### 1. Introduction
We will now put oxygen in different places and configurations on the platinum surface. For this, identify the adsorption sites as shown in Figure 1. These sites can directly be used for adsorption of atomic O. 

<img src="images/pt-111.png"  width="300">

Figure 1: Top view of the adsorption sites on a {111} face-centered cubic surface

### 2. Dissociative adsorption
Place the O atoms about 1Å above the surface sites. Create the following calculations:
* O at the top site
* O at the bridge site
* O at the fcc hollow site
* O at the hcp hollow site

In the atomic structure, we kept the bottom two Pt layers fixed at their bulk positions, which is an (admittedly pretty drastic) approximation of the surface of a very large solid.

Each calculation will take a long time (10~20 min), therefore we will not run the calculation one by one using a normal `for` loop. Instead, a `mypool` function was used to sumbit all the calculations at the same time, thus all jobs will run parallelly. 

#todo: set the ecutwfc and kpts by yourself.

In [ ]:
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from ase.atoms import Atoms
from xespresso import Espresso
from xespresso.tools import mypool
from time import sleep


pseudopotentials = {
    'O': 'O.pbe-n-rrkjus_psl.1.0.0.UPF',
    'Pt': 'Pt.pbe-n-rrkjus_psl.1.0.0.UPF',}
# The calculation will take long time, thus please ask your assistant for suggesition on which queue should be used.
queue = {'ntasks': 40, 'partition': 'empi', 'time': '00:59:00'}
# queue = {'ntasks': 16, 'partition': 'all', 'account': 'dcb', 'time': '00:59:00'}

#-----------------------------------------------------------------------
# build Pt surface
pt = fcc111('Pt', size=(2, 2, 3), vacuum=6.0)
constraint = FixAtoms(mask=[atom.tag >= 2 for atom in pt])
pt.set_constraint(constraint)
#-----------------------------------------------------------------------
# add oxygen atoms on platinum surfaces, and save the structures into dissociative_jobs
o = Atoms('O')
dissociative_jobs = {}
for site in ['ontop', 'bridge', 'fcc', 'hcp']:
    atoms = pt.copy()
    ads = o.copy()
    add_adsorbate(atoms, ads, height=1.0, position=site)
    dissociative_jobs[site] = atoms
#-----------------------------------------------------------------------
# def a function to run a calculation for one site
def run(site, atoms, t):
    sleep(t)    # let the code sleep for a random time (in few seconds), to avoid all jobs run at exact time (traffic jam), which is bad for the high performace computing center.
    calc = Espresso(label = 'surface/pt-o-{0}'.format(site),
                    pseudopotentials = pseudopotentials,
                    calculation = 'relax',   # allow atoms to move
                    ecutwfc = 40,
                    occupations = 'smearing',
                    degauss = 0.01,
                    nspin = 2,
                    input_data = {'input_ntyp':{'starting_magnetization': {'O': 1.0}}},
                    kpts = (6, 6, 1),
                    queue = queue,
                    parallel = '-nk 4')
    atoms.calc = calc
    atoms.get_potential_energy()
    e = atoms.calc.results['energy']
    print('{0} {1:3.3f}'.format(site, e))
#-----------------------------------------------------------------------
# submit the calculation for all site at the same time
mypool(dissociative_jobs, run)



### 3. Molecular adsorption
Molecular O$_2$ will adsorb in a geometry where two oxygen atoms are bound to the surface sites, here we will try two most stable geometry: 'top-top' and 'top-fcc'. (we have found those by trying lots of configurations, which we will skip here to not make the practical too long).

Let's look at the platinum surface again and chose approximate positions for the oxygen atoms.

In [ ]:
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from ase.atoms import Atoms
from x3dase.visualize import view_x3d_n
# look at the platinum surface
pt = fcc111('Pt', size=(2, 2, 3), vacuum=6.0)
constraint = FixAtoms(mask=[atom.tag >= 2 for atom in pt])
pt.set_constraint(constraint)
view_x3d_n(pt, bond = 1.0, label = True, output = 'htmls/pt-o2.html')

Now let's add two oxygen atoms on the 'top-top' and 'top-fcc' sites., #todo: build the atomic structure, `atoms = atoms + o1 + o2`

Please measure the distance between the two oxygen atoms, and make sure the two oxygen atoms are not far away from each other, because they are still forming one O$_2$ molecule.

In [ ]:
from x3dase.visualize import view_x3d_n
#todo buid the structure of O2 adsorption, chose approximate positions for the oxygen atom close to top site.
# Our calculation will find the accurate position.

molecular_jobs = {}
#====================================================
# top-top
atoms = pt.copy()
# add the second one, please set up a position, e.g. [0.5, 0.5, 12.5]
o1 = Atoms('O', positions = [[?, ?, ?]])     # 0.6, 0.7, 12.5
o2 = Atoms('O', positions = [[?, ?, ?]])     # 1.2, 1.9, 12.5
atoms = atoms + o1 + o2
molecular_jobs['top-top'] = atoms.copy()
view_x3d_n(atoms, bond = 1.0, label = True, output = 'htmls/pt-o2-top-top.html')

In [ ]:
#====================================================
# top-fcc
atoms = pt.copy()
# add the second one, please set up a position, e.g. [0.5, 0.5, 12.5]
o1 = Atoms('O', positions = [[?, ?, ?]])    # 0.7, 0.5, 12.3
o2 = Atoms('O', positions = [[?, ?, ?]])    # 1.8, 1.0, 12.0
atoms = atoms + o1 + o2
molecular_jobs['top-fcc'] = atoms.copy()
view_x3d_n(atoms, bond = 1.0, label = True, output = 'htmls/pt-o2-top-fcc.html')

In [ ]:
from xespresso import Espresso
from xespresso.tools import mypool
from time import sleep

pseudopotentials = {
    'O': 'O.pbe-n-rrkjus_psl.1.0.0.UPF',
    'Pt': 'Pt.pbe-n-rrkjus_psl.1.0.0.UPF',}

queue = {'ntasks': 20, 'partition': 'empi', 'time': '5:59:00'}           

molecular_energies = {}
# def a function to run a calculation for one site
def run(site, atoms, t):
    sleep(t)    # let the code sleep for a random time (in few seconds), to avoid all jobs run at exact time (traffic jam), which is bad for the high performace computing center.
    calc = Espresso(label = 'surface/pt-o2-{0}'.format(site),
                    pseudopotentials = pseudopotentials,
                    calculation = 'relax',   # allow atoms to move
                    ecutwfc = 40,
                    occupations = 'smearing',
                    degauss = 0.01,
                    nspin = 2,
                    input_data = {'input_ntyp':{'starting_magnetization': {'O': 1.0}}},
                    mixing_mode = 'local-TF',
                    mixing_beta = 0.5,
                    electron_maxstep = 200,
                    kpts = (6, 6, 1),
                    queue = queue,
                    parallel = '-nk 4')
    atoms.calc = calc
    atoms.get_potential_energy()
    e = atoms.calc.results['energy']
    print('{0} {1:3.3f}'.format(site, e))
    molecular_energies[site] = e
# submit the calculation for all site at the same time
mypool(molecular_jobs, run)
# todo get the magmom on oxygen atoms

Does the O$_2$ molecule still have the triplet ground state when adsorbed on the surface? What does this tell you about charge transfers and the charge state of the O$_2$ molecule? Measure the O-O bond length and compare it to the one in O$_2$, O$_2$$^-$ (superoxide) and O$_2$$^{2-}$ (peroxide).

### 4. Calculation the adsorption energy
The adsorption energy is given by

$\Delta E_{ads} = E_{surf + ads} - (E_{surf} + E_{ads})$

where $E_{surf + ads}$ is the energy of the surface with the adsorbate (O or O$_2$), $E_{surf}$ is the energy of the clean surface and $E_{ads}$ is the energy of the adsorbate, which we take as the energy of O$_2$ molecule for the case of O$_2$ and half the energy of O$_2$ molecule for the case of O. A negative value means that there is an energy gain for adsorption, while a positive value means that adsorption is not energetically favorable.

Is O adsorption energetically favored to O$_2$ formation at all sites? If you have sites with a positive adsorption energy (unfavorable adsorption), exclude them from further analysis. Do all adsorbed species stay at the sites where you put them (similar adsorption energies can hint at same final sites). If you have same sites, discard the ones that did not stay at their initial site from further analysis.

In [ ]:
#todo, please use the energies you got from the previous calculation
# Dissociative adsorption
E_pt = ?                  # -10665.115
E_o2 = ?                  # -877.827
dissociative_energies = {
'ontop': ?,               # -11103.908
'bridge': ?,              # -11104.887
'fcc': ?,                 # -11105.277
'hcp': ?,                 # -11104.802
}
print('Sites     Adsorption energies (eV)')
for site, E_ads in dissociative_energies.items():
    dE_ads = E_ads - (E_pt + E_o2/2.0)
    print('{0:10s}  {1:1.2f}'.format(site, dE_ads))


In [ ]:
#=====================================================
#  Molecular adsorption
molecular_energies = {
'top-top': ?,                # -11543.680,
'top-fcc': ?,                # -11543.638,
}
print('Sites     Adsorption energies (eV)')
for site, E_ads in molecular_energies.items():
    dE_ads = E_ads - (E_pt + E_o2)
    print('{0:10s}  {1:1.2f}'.format(site, dE_ads))

### 5. Calculate the desorption temperature
The desorption temperature $T$ can be get by solving this equation, please read the description for detail.

$\frac{E_a}{kT^2} - \frac{xvN^{x-1}}{\beta}exp(\frac{-E_a}{kT}) = 0       \tag{1}          $

For molecular adsorption (O$_2$), we know that the desorption process is of first order ($x$ = 1) with an attempt frequency around the vibration frequency of the adsorbed O$_2$ molecule, which can reasonably be assumed to be around $v = 10^{13}$ Hz). 

Use your obtained adsorption energy for O$_2$ together with equation (1) in Python nonlinear solver to get the corresponding desorption temperature (pay attention to units). Is this adsorption energy consistent with the data for 0.25 ML in Figure 3? (The description document) What can we conclude regarding the fact if O$_2$ is adsorbed molecularly or dissociatively on Pt (111)? Does the attempt frequency, which we grossly approximated have a significant effect on the result?

In [ ]:
from ase.units import kB      # unit eV K-1
from math import exp
from scipy import optimize

x = 1
v = 10**13     # unit Hz, s-1
beta = 6      # K s-1
N = 0.25
Ea = '#todo'     # unit eV

def Func(T):
    # T unit K
    return Ea/kB/T**2 - x*v*N**(x-1)/beta*exp(-Ea/kB/T)


T = optimize.broyden1(Func, 1000, f_tol=1e-14)

print('Desorption temperature: ', T)


### 6. Calibrating the model for dissociative adsorption
For dissociative adsorption, we have a second order process and we can unfortunately not easily estimate the attempt frequency from molecular vibrations. We will therefore make the assumption that the 0.25 ML peak in Figure 3 stems from desorption of dissociated O$_2$ molecules. Using your **most favorable** calculated adsorption energy, use equation 6 to determine the attempt frequency for the given experimental parameters ($N$=0.25, $\beta$=6K/s). 

In [ ]:
from ase.units import kB      # unit eV K-1
from math import exp
from scipy import optimize

x = 2
beta = 6      # K s-1
N = 0.25
Ea = '#todo'     # unit eV

T = 850
print(Ea/kB/T**2, x*v*N**(x-1)/beta*exp(-Ea/kB/T))
def calc_v(T):
    # T unit K
    v = (Ea/kB/T**2) / (x*N**(x-1)/beta*exp(-Ea/kB/T))
    return v

v1 = calc_v(850)

print('Attempt frequency: {0:1.2} Hz'.format(v1))


### 7. Explaining the experimental TPD spectrum
Using this attempt frequency obtained in the last point, use a nonlinear solver to obtain the desorption peak temperature for the **second most favorable** adsorption site. How well does this peak temperature agree with experiment? What physical model do we assume for this process and is this realistic (you may want to discuss this with your assistant).

What would a more realistic model be? Construct an adsorption energy for this case and use it with the calculated attempt frequency to predict the peak temperature. Does this fit the experimental data better? What do you hence assign the different TPD peaks to? Think about the fact that we used a coverage-independent adsorption energy (at 0.25 ML). Typically, adsorption gets stronger at lower coverages. How would you expect this to affect the peak positions? 

In [ ]:
from ase.units import kB      # unit eV K-1
from math import exp
from scipy import optimize

x = 2
v = '#todo'     # unit Hz, s-1
beta = 6      # K s-1
N = 0.25
Ea = ?     # unit eV

def Func(T):
    # T unit K
    return Ea/kB/T**2 - x*v*N**(x-1)/beta*exp(-Ea/kB/T)

T = optimize.broyden1(Func, 800, f_tol=1e-14)

print('Desorption temperature: ', T)
